In [1]:
# Import the modules
import numpy as np
import pandas as pd
from scipy.spatial import distance
from IPython.display import Image
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
# from sklearn.preprocessing import StandardScaler
from joblib import dump

In [2]:
!pip install meteostat
from meteostat import Point, Daily

In [3]:
#  Import and read the charity_data.csv.
movies_df = pd.read_csv("./Resources/movies.csv")
movies_df


,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,NaN,Drama,2020,"October 23, 2020 (United States)",3.1,18.0,Joseph Ebanks,Joseph Ebanks,Shannon Bond,United States,7000.0,NaN,NaN,90.0
7664,Dream Round,NaN,Comedy,2020,"February 7, 2020 (United States)",4.7,36.0,Dusty Dukatz,Lisa Huston,Michael Saquella,United States,NaN,NaN,Cactus Blue Entertainment,90.0
7665,Saving Mbango,NaN,Drama,2020,"April 27, 2020 (Cameroon)",5.7,29.0,Nkanya Nkwai,Lynno Lovert,Onyama Laura,United States,58750.0,NaN,Embi Productions,NaN
7666,It's Just Us,NaN,Drama,2020,"October 1, 2020 (United States)",NaN,NaN,James Randall,James Randall,Christina Roz,United States,15000.0,NaN,NaN,120.0


In [13]:
movies_df.reset_index

<bound method DataFrame.reset_index of                                                 name rating      genre  year  \
0                                        The Shining      R      Drama  1980   
1                                    The Blue Lagoon      R  Adventure  1980   
2     Star Wars: Episode V - The Empire Strikes Back     PG     Action  1980   
3                                          Airplane!     PG     Comedy  1980   
4                                         Caddyshack      R     Comedy  1980   
...                                              ...    ...        ...   ...   
7663                                    More to Life    NaN      Drama  2020   
7664                                     Dream Round    NaN     Comedy  2020   
7665                                   Saving Mbango    NaN      Drama  2020   
7666                                    It's Just Us    NaN      Drama  2020   
7667                                       Tee em el    NaN     Horror  2020   



In [5]:
movies_df['released']

0          June 13, 1980 (United States)
1           July 2, 1980 (United States)
2          June 20, 1980 (United States)
3           July 2, 1980 (United States)
4          July 25, 1980 (United States)
                      ...               
7663    October 23, 2020 (United States)
7664    February 7, 2020 (United States)
7665           April 27, 2020 (Cameroon)
7666     October 1, 2020 (United States)
7667     August 19, 2020 (United States)
Name: released, Length: 7668, dtype: object

In [6]:
movie_monthdf = movies_df["released"].str.split(" ", expand = True)
movie_monthdf.reset_index
movie_monthdf

,0,1,2,3,4,5,6
0,June,"13,",1980,(United,States),None,None
1,July,"2,",1980,(United,States),None,None
2,June,"20,",1980,(United,States),None,None
3,July,"2,",1980,(United,States),None,None
4,July,"25,",1980,(United,States),None,None
...,...,...,...,...,...,...,...
7663,October,"23,",2020,(United,States),None,None
7664,February,"7,",2020,(United,States),None,None
7665,April,"27,",2020,(Cameroon),None,None,None
7666,October,"1,",2020,(United,States),None,None


In [ ]:
#movies_df["released"] = movies_df["released"].str.split(" ").astype('string')
#movies_df

In [9]:
new_movie_df = movie_monthdf.drop(movie_monthdf.loc[:, 1:6], axis = 1)
new_movie_df

,0
0,June
1,July
2,June
3,July
4,July
...,...
7663,October
7664,February
7665,April
7666,October


In [10]:
movie_df2 = new_movie_df.rename(columns = {0: 'month'})
movie_df3 = movie_df2.reset_index()
movie_df3

,index,month
0,0,June
1,1,July
2,2,June
3,3,July
4,4,July
...,...,...
7663,7663,October
7664,7664,February
7665,7665,April
7666,7666,October


In [11]:
combo_df = pd.merge(movie_index, movie_df3, on=['index'],how='outer')
combo_df

NameError: name 'movie_index' is not defined